In [11]:
from datasets import load_dataset
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [14]:
data = load_dataset("csv", data_files="../raw_data/influence_of_training_data.csv", split="train")

Using custom data configuration default-78fe9711fdeffc1c
Reusing dataset csv (/home/teven/.cache/huggingface/datasets/csv/default-78fe9711fdeffc1c/0.0.0/50d3f5d1aef1732094f95566e0c67506165deea4ff693f78b1a17588fb6bc108)


In [26]:
data = data.map(None, remove_columns=[column_name for column_name in data.column_names if "_step" in column_name or "MIN" in column_name or "MAX" in column_name])

  0%|          | 0/103 [00:00<?, ?ex/s]

In [29]:
data.column_names

['data_points',
 'trex_bs192_hardneg - WebNLG_recall@1',
 'tekgen_bs192_hardneg - WebNLG_recall@1',
 'kelm_bs192_hardneg - WebNLG_recall@1',
 'all_bs192_hardneg - WebNLG_recall@1',
 'trex_bs320_batchneg - WebNLG_recall@1',
 'tekgen_bs320_batchneg - WebNLG_recall@1',
 'kelm_bs320_batchneg - WebNLG_recall@1',
 'all_bs320_batchneg - WebNLG_recall@1',
 'trex_bs2560_hardneg - WebNLG_recall@1',
 'tekgen_bs2560_hardneg - WebNLG_recall@1',
 'kelm_bs2560_hardneg - WebNLG_recall@1',
 'all_bs2560_hardneg - WebNLG_recall@1',
 'baseline - WebNLG_recall@1']

In [30]:
models = [model_name.split()[0] for model_name in data.column_names[1:]]

In [31]:
models

['trex_bs192_hardneg',
 'tekgen_bs192_hardneg',
 'kelm_bs192_hardneg',
 'all_bs192_hardneg',
 'trex_bs320_batchneg',
 'tekgen_bs320_batchneg',
 'kelm_bs320_batchneg',
 'all_bs320_batchneg',
 'trex_bs2560_hardneg',
 'tekgen_bs2560_hardneg',
 'kelm_bs2560_hardneg',
 'all_bs2560_hardneg',
 'baseline']

In [ ]:
performance_per_model = {}